## PRABAL GHOSH         AM.SC.P2CSC20040       ASSIGNMENT-6_(Fine_tune_BERT)

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
     |████████████████████████████████| 895 kB 68.7 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [3]:
import tensorflow as tf

In [10]:
from google.colab import files
uploaded = files.upload()

Saving IMDB Dataset_validate.csv.zip to IMDB Dataset_validate.csv.zip


https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?select=IMDB+Dataset.csv

In [11]:
df = pd.read_csv('IMDB Dataset_validate.csv.zip',compression='zip')


In [13]:
df.head(10)

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0
5,This is the best movie I've ever seen! <br /><...,1
6,The morbid Catholic writer Gerard Reve (Jeroen...,1
7,"""Semana Santa"" or ""Angel Of Death"" is a very w...",0
8,Somebody mastered the difficult task of mergin...,1
9,Why did I waste 1.5 hours of my life watching ...,0


In [14]:
reviews = df['text'].values.tolist()
labels = df['label'].tolist()

In [18]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels=train_test_split(reviews,labels,test_size=0.33, random_state=4)

In [20]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [21]:
tokenizer([training_sentences[0]], truncation=True,padding=True, max_length=128)

{'input_ids': [[101, 1996, 2143, 2003, 15703, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 10892, 1010, 2045, 2024, 2205, 2116, 2335, 2017, 2156, 4895, 14876, 7874, 2098, 1998, 2200, 5560, 5493, 5019, 1012, 2028, 2071, 4089, 2131, 1037, 20133, 2143, 2478, 1037, 11519, 5515, 4950, 1998, 2531, 1002, 2678, 9260, 4007, 1012, 2091, 2000, 3011, 1010, 2158, 2006, 1996, 2395, 2987, 1005, 1056, 2812, 28810, 9260, 1012, 4895, 14876, 7874, 2098, 5019, 2008, 2123, 1005, 1056, 5383, 2590, 4861, 2323, 2031, 2042, 17159, 1012, 1996, 2168, 3632, 2005, 2437, 2469, 2008, 1996, 4874, 1005, 1055, 2132, 1013, 2192, 1013, 2500, 2994, 1999, 1996, 4853, 1012, 2026, 1022, 2086, 2214, 2365, 4282, 2008, 2011, 2085, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2143, 2003, 2126, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [22]:
train_encodings = tokenizer(training_sentences,
truncation=True,
padding=True)
val_encodings = tokenizer(validation_sentences,
truncation=True,
padding=True)
train_dataset = tf.data.Dataset.from_tensor_slices((
dict(train_encodings),
training_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
dict(val_encodings),
validation_labels 
))

In [39]:
model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=6)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use i

In [46]:
test_sentence = "This is a really bad Movie. I hate it"
predict_input = tokenizer.encode(test_sentence,
truncation=True,
padding=True,
return_tensors="tf")

In [47]:
model.save_pretrained("/content/drive/MyDrive")

In [48]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive")

Some layers from the model checkpoint at /content/drive/MyDrive were not used when initializing TFDistilBertForSequenceClassification: ['dropout_99']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive and are newly initialized: ['dropout_139']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

Negative
